In [7]:
import os
import numpy as np
from dnnbrain.dnn.core import Stimulus
from dnnbrain.utils.util import gen_dmask
from dnnbrain.dnn import models as db_models
import torch.nn as nn
import torch
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from dnnbrain.io.fileio import ActivationFile
from activation_PSI import Dnn_act, sparseness

In [8]:
# specify custom paremeters
root = os.getcwd()  # root directory of analysis
net_model = 'AlexNet'  # ['AlexNet', 'Vgg11']
permuted_method = 'permut_weight' # ['permut_weight','permut_bias']
n = 1  # number of permuted models

stim_path = os.path.join(root, 'imagenet.stim.csv') # make sure to change the path in [dataset].stim.csv to the local directory of the dataset 

In [9]:
# result path of the specified net
net = net_model.lower() + '_' + permuted_method
net_dir = os.path.join(root, net)
if os.path.exists(net_dir) is False:
    os.mkdir(net_dir)
stim_per_cat = 50

# Load DNN, stimuli and define ablated models
dnn = eval('db_models.{}()'.format(net_model))  # load DNN
stimuli = Stimulus()
stimuli.load(stim_path) # load stimuli

# generate DNN mask
layer_name = [dnn.layers[i] for i in range(len(dnn.layers)) if dnn.layers[i].split('_')[-1] == 'relu']
dmask = gen_dmask(layer_name)  

# save dir
out_dir = os.path.join(net_dir,'dnn_activation')
if os.path.exists(out_dir) is not True:
    os.makedirs(out_dir)

In [10]:
# permut weights pretrained network for each layer, finest granularity purmutation    
def permut_weight_model(dnn):
    new_dict = {}
    for key in dnn.model.state_dict().keys():
        if key.split('.')[-1] == 'weight':
            para = dnn.model.state_dict()[key]
            ori_shape = para.shape
            new_dict[key] = para.view(-1)[
                    torch.randperm(*para.view(-1).size())].reshape(ori_shape)
        else:
            new_dict[key] = dnn.model.state_dict()[key]
        
    dnn.model.load_state_dict(new_dict)

# permut bias pretrained network for each layer
def permut_bias_model(dnn):
    new_dict = {}
    for key in dnn.model.state_dict().keys():
        if key.split('.')[-1] == 'bias':
            para = dnn.model.state_dict()[key]
            ori_shape = para.shape
            new_dict[key] = para.view(-1)[
                    torch.randperm(*para.view(-1).size())].reshape(ori_shape)
        else:
            new_dict[key] = dnn.model.state_dict()[key]
        
    dnn.model.load_state_dict(new_dict)

In [12]:
# extract activation from n permuted models
for i in range(n):   

    # prepare model
    if permuted_method == 'permut_weight':       
        permut_weight_model(dnn)
    elif permuted_method == 'permut_bias':       
        permut_bias_model(dnn)

    # extract activation-
    activation = dnn.compute_activation(stimuli, dmask, 'mean', cuda=True)

    # To save storage, compute category-wise mean and replace the original image-wise act
    for layer in list(activation.layers):
        cur_shape = np.shape(activation.get(layer))
        new_shape = np.r_[int(cur_shape[0] / stim_per_cat), 
                          stim_per_cat, cur_shape[1:]]
        cat_data = activation.get(layer).reshape(new_shape)
        activation.set(layer, cat_data.mean(1))

    # save act
    out_path = os.path.join(out_dir, '{0}_imagenet_{1}.act.h5'.format(net.lower(), i))
    activation.save(out_path)


Extracted activation of conv1_relu: 8/50000
Extracted activation of conv1_relu: 16/50000
Extracted activation of conv1_relu: 24/50000
Extracted activation of conv1_relu: 32/50000
Extracted activation of conv1_relu: 40/50000
Extracted activation of conv1_relu: 48/50000
Extracted activation of conv1_relu: 56/50000
Extracted activation of conv1_relu: 64/50000
Extracted activation of conv1_relu: 72/50000
Extracted activation of conv1_relu: 80/50000
Extracted activation of conv1_relu: 88/50000
Extracted activation of conv1_relu: 96/50000
Extracted activation of conv1_relu: 104/50000
Extracted activation of conv1_relu: 112/50000
Extracted activation of conv1_relu: 120/50000
Extracted activation of conv1_relu: 128/50000
Extracted activation of conv1_relu: 136/50000
Extracted activation of conv1_relu: 144/50000
Extracted activation of conv1_relu: 152/50000
Extracted activation of conv1_relu: 160/50000
Extracted activation of conv1_relu: 168/50000
Extracted activation of conv1_relu: 176/50000
E

Extracted activation of conv1_relu: 1432/50000
Extracted activation of conv1_relu: 1440/50000
Extracted activation of conv1_relu: 1448/50000
Extracted activation of conv1_relu: 1456/50000
Extracted activation of conv1_relu: 1464/50000
Extracted activation of conv1_relu: 1472/50000
Extracted activation of conv1_relu: 1480/50000
Extracted activation of conv1_relu: 1488/50000
Extracted activation of conv1_relu: 1496/50000
Extracted activation of conv1_relu: 1504/50000
Extracted activation of conv1_relu: 1512/50000
Extracted activation of conv1_relu: 1520/50000
Extracted activation of conv1_relu: 1528/50000
Extracted activation of conv1_relu: 1536/50000
Extracted activation of conv1_relu: 1544/50000
Extracted activation of conv1_relu: 1552/50000
Extracted activation of conv1_relu: 1560/50000
Extracted activation of conv1_relu: 1568/50000
Extracted activation of conv1_relu: 1576/50000
Extracted activation of conv1_relu: 1584/50000
Extracted activation of conv1_relu: 1592/50000
Extracted act

Extracted activation of conv1_relu: 2832/50000
Extracted activation of conv1_relu: 2840/50000
Extracted activation of conv1_relu: 2848/50000
Extracted activation of conv1_relu: 2856/50000
Extracted activation of conv1_relu: 2864/50000
Extracted activation of conv1_relu: 2872/50000
Extracted activation of conv1_relu: 2880/50000
Extracted activation of conv1_relu: 2888/50000
Extracted activation of conv1_relu: 2896/50000
Extracted activation of conv1_relu: 2904/50000
Extracted activation of conv1_relu: 2912/50000
Extracted activation of conv1_relu: 2920/50000
Extracted activation of conv1_relu: 2928/50000
Extracted activation of conv1_relu: 2936/50000
Extracted activation of conv1_relu: 2944/50000
Extracted activation of conv1_relu: 2952/50000
Extracted activation of conv1_relu: 2960/50000
Extracted activation of conv1_relu: 2968/50000
Extracted activation of conv1_relu: 2976/50000
Extracted activation of conv1_relu: 2984/50000
Extracted activation of conv1_relu: 2992/50000
Extracted act

Extracted activation of conv1_relu: 4240/50000
Extracted activation of conv1_relu: 4248/50000
Extracted activation of conv1_relu: 4256/50000
Extracted activation of conv1_relu: 4264/50000
Extracted activation of conv1_relu: 4272/50000
Extracted activation of conv1_relu: 4280/50000
Extracted activation of conv1_relu: 4288/50000
Extracted activation of conv1_relu: 4296/50000
Extracted activation of conv1_relu: 4304/50000
Extracted activation of conv1_relu: 4312/50000
Extracted activation of conv1_relu: 4320/50000
Extracted activation of conv1_relu: 4328/50000
Extracted activation of conv1_relu: 4336/50000
Extracted activation of conv1_relu: 4344/50000
Extracted activation of conv1_relu: 4352/50000
Extracted activation of conv1_relu: 4360/50000
Extracted activation of conv1_relu: 4368/50000
Extracted activation of conv1_relu: 4376/50000
Extracted activation of conv1_relu: 4384/50000
Extracted activation of conv1_relu: 4392/50000
Extracted activation of conv1_relu: 4400/50000
Extracted act

Extracted activation of conv1_relu: 5640/50000
Extracted activation of conv1_relu: 5648/50000
Extracted activation of conv1_relu: 5656/50000
Extracted activation of conv1_relu: 5664/50000
Extracted activation of conv1_relu: 5672/50000
Extracted activation of conv1_relu: 5680/50000
Extracted activation of conv1_relu: 5688/50000
Extracted activation of conv1_relu: 5696/50000
Extracted activation of conv1_relu: 5704/50000
Extracted activation of conv1_relu: 5712/50000
Extracted activation of conv1_relu: 5720/50000
Extracted activation of conv1_relu: 5728/50000
Extracted activation of conv1_relu: 5736/50000
Extracted activation of conv1_relu: 5744/50000
Extracted activation of conv1_relu: 5752/50000
Extracted activation of conv1_relu: 5760/50000
Extracted activation of conv1_relu: 5768/50000
Extracted activation of conv1_relu: 5776/50000
Extracted activation of conv1_relu: 5784/50000
Extracted activation of conv1_relu: 5792/50000
Extracted activation of conv1_relu: 5800/50000
Extracted act

...

Extracted activation of fc2_relu: 43352/50000
Extracted activation of fc2_relu: 43360/50000
Extracted activation of fc2_relu: 43368/50000
Extracted activation of fc2_relu: 43376/50000
Extracted activation of fc2_relu: 43384/50000
Extracted activation of fc2_relu: 43392/50000
Extracted activation of fc2_relu: 43400/50000
Extracted activation of fc2_relu: 43408/50000
Extracted activation of fc2_relu: 43416/50000
Extracted activation of fc2_relu: 43424/50000
Extracted activation of fc2_relu: 43432/50000
Extracted activation of fc2_relu: 43440/50000
Extracted activation of fc2_relu: 43448/50000
Extracted activation of fc2_relu: 43456/50000
Extracted activation of fc2_relu: 43464/50000
Extracted activation of fc2_relu: 43472/50000
Extracted activation of fc2_relu: 43480/50000
Extracted activation of fc2_relu: 43488/50000
Extracted activation of fc2_relu: 43496/50000
Extracted activation of fc2_relu: 43504/50000
Extracted activation of fc2_relu: 43512/50000
Extracted activation of fc2_relu: 

Extracted activation of fc2_relu: 44808/50000
Extracted activation of fc2_relu: 44816/50000
Extracted activation of fc2_relu: 44824/50000
Extracted activation of fc2_relu: 44832/50000
Extracted activation of fc2_relu: 44840/50000
Extracted activation of fc2_relu: 44848/50000
Extracted activation of fc2_relu: 44856/50000
Extracted activation of fc2_relu: 44864/50000
Extracted activation of fc2_relu: 44872/50000
Extracted activation of fc2_relu: 44880/50000
Extracted activation of fc2_relu: 44888/50000
Extracted activation of fc2_relu: 44896/50000
Extracted activation of fc2_relu: 44904/50000
Extracted activation of fc2_relu: 44912/50000
Extracted activation of fc2_relu: 44920/50000
Extracted activation of fc2_relu: 44928/50000
Extracted activation of fc2_relu: 44936/50000
Extracted activation of fc2_relu: 44944/50000
Extracted activation of fc2_relu: 44952/50000
Extracted activation of fc2_relu: 44960/50000
Extracted activation of fc2_relu: 44968/50000
Extracted activation of fc2_relu: 

Extracted activation of fc2_relu: 46272/50000
Extracted activation of fc2_relu: 46280/50000
Extracted activation of fc2_relu: 46288/50000
Extracted activation of fc2_relu: 46296/50000
Extracted activation of fc2_relu: 46304/50000
Extracted activation of fc2_relu: 46312/50000
Extracted activation of fc2_relu: 46320/50000
Extracted activation of fc2_relu: 46328/50000
Extracted activation of fc2_relu: 46336/50000
Extracted activation of fc2_relu: 46344/50000
Extracted activation of fc2_relu: 46352/50000
Extracted activation of fc2_relu: 46360/50000
Extracted activation of fc2_relu: 46368/50000
Extracted activation of fc2_relu: 46376/50000
Extracted activation of fc2_relu: 46384/50000
Extracted activation of fc2_relu: 46392/50000
Extracted activation of fc2_relu: 46400/50000
Extracted activation of fc2_relu: 46408/50000
Extracted activation of fc2_relu: 46416/50000
Extracted activation of fc2_relu: 46424/50000
Extracted activation of fc2_relu: 46432/50000
Extracted activation of fc2_relu: 

Extracted activation of fc2_relu: 47704/50000
Extracted activation of fc2_relu: 47712/50000
Extracted activation of fc2_relu: 47720/50000
Extracted activation of fc2_relu: 47728/50000
Extracted activation of fc2_relu: 47736/50000
Extracted activation of fc2_relu: 47744/50000
Extracted activation of fc2_relu: 47752/50000
Extracted activation of fc2_relu: 47760/50000
Extracted activation of fc2_relu: 47768/50000
Extracted activation of fc2_relu: 47776/50000
Extracted activation of fc2_relu: 47784/50000
Extracted activation of fc2_relu: 47792/50000
Extracted activation of fc2_relu: 47800/50000
Extracted activation of fc2_relu: 47808/50000
Extracted activation of fc2_relu: 47816/50000
Extracted activation of fc2_relu: 47824/50000
Extracted activation of fc2_relu: 47832/50000
Extracted activation of fc2_relu: 47840/50000
Extracted activation of fc2_relu: 47848/50000
Extracted activation of fc2_relu: 47856/50000
Extracted activation of fc2_relu: 47864/50000
Extracted activation of fc2_relu: 

Extracted activation of fc2_relu: 49168/50000
Extracted activation of fc2_relu: 49176/50000
Extracted activation of fc2_relu: 49184/50000
Extracted activation of fc2_relu: 49192/50000
Extracted activation of fc2_relu: 49200/50000
Extracted activation of fc2_relu: 49208/50000
Extracted activation of fc2_relu: 49216/50000
Extracted activation of fc2_relu: 49224/50000
Extracted activation of fc2_relu: 49232/50000
Extracted activation of fc2_relu: 49240/50000
Extracted activation of fc2_relu: 49248/50000
Extracted activation of fc2_relu: 49256/50000
Extracted activation of fc2_relu: 49264/50000
Extracted activation of fc2_relu: 49272/50000
Extracted activation of fc2_relu: 49280/50000
Extracted activation of fc2_relu: 49288/50000
Extracted activation of fc2_relu: 49296/50000
Extracted activation of fc2_relu: 49304/50000
Extracted activation of fc2_relu: 49312/50000
Extracted activation of fc2_relu: 49320/50000
Extracted activation of fc2_relu: 49328/50000
Extracted activation of fc2_relu: 

In [13]:
# load activation
n = 1 # number of permuted models
stim_per_cat = 1
sp_median = []
for i in range(n):
    dnnact_path = os.path.join(out_dir, '{0}_imagenet_{1}.act.h5'.format(net.lower(), i))
    dnnact_alllayer = ActivationFile(dnnact_path).read()
    layer_name = list(dnnact_alllayer.keys())
    
    sp = []
    for layer in layer_name:
    
        dnnact = Dnn_act(dnnact_alllayer[layer], stim_per_cat=stim_per_cat)       
        dnnact_catmean = dnnact.cat_mean_act()[0][:, :, 0]
        
        dnnact_catmean_z = np.nan_to_num(stats.zscore(dnnact_catmean, 0))
    
        # population sparseness
        sparse_p = sparseness(dnnact_catmean_z.T, type='s', norm=True)

        sp.append(np.squeeze(sparse_p))    
        print('{0} done'.format(layer))

    sp_median.append(np.array([np.nanmedian(sp[i]) for i in range(len(sp))]))

sp_median = np.asarray(sp_median).T

conv1_relu done
conv2_relu done
conv3_relu done
conv4_relu done
conv5_relu done


/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: divide by zero encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: divide by zero encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


fc1_relu done
fc2_relu done


/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: divide by zero encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: divide by zero encountered in true_divide
  return (a - mns) / sstd
/usr/local/neurosoft/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [ ]:
# plot Fig 3B & D
plt.boxplot(sp_median.T)